In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
%load_ext nb_black


<IPython.core.display.Javascript object>

In [6]:
import tensorflow as tf
import tensorflow_gnn as tfgnn

        # Model hyper-parameters:
        h_dims = {'user': 256, 'movie': 64, 'genre': 128}

        # Model builder initialization:
        gnn = tfgnn.keras.ConvGNNBuilder(
          lambda edge_set_name: WeightedSumConvolution(),
          lambda node_set_name: tfgnn.keras.layers.NextStateFromConcat(
             tf.keras.layers.Dense(h_dims[node_set_name]))
        )

        # Two rounds of message passing to target node sets:
        model = tf.keras.models.Sequential([
            gnn.Convolve({'genre'}),  # sends messages from movie to genre
            gnn.Convolve({'user'}),  # sends messages from movie and genre to users
            tfgnn.keras.layers.Readout(node_set_name="user"),
            tf.keras.layers.Dense(1)
        ])

IndentationError: unexpected indent (3212825241.py, line 2)

ERROR:root:Cannot parse: 2:0:     import tensorflow_gnn as tfgnn
Traceback (most recent call last):
  File "C:\Users\psraj\miniconda3\envs\m_l\lib\site-packages\lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "C:\Users\psraj\miniconda3\envs\m_l\lib\site-packages\lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "src\black\__init__.py", line 1079, in format_str
  File "src\black\__init__.py", line 1089, in _format_str_once
  File "src\black\parsing.py", line 127, in lib2to3_parse
black.parsing.InvalidInput: Cannot parse: 2:0:     import tensorflow_gnn as tfgnn
